# environment set up

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
!pip install deep_sort_realtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 38.4 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
# /content/drive/MyDrive/icebergtrack/deep_sort/deep_sort
sys.path.append('/content/drive/MyDrive/icebergtrack/deep_sort/')

In [3]:
sys.path.append('/content/drive/MyDrive/icebergtrack/deep_sort/tools')
import generate_detections as gdet

In [4]:
import numpy as np

In [5]:
sys.path.append('/content/drive/MyDrive/icebergtrack/deep_sort/deep_sort')
from tracker import Tracker as DeepSortTracker
import nn_matching
from detection import Detection

In [6]:
import os
import random
import cv2
import glob
import numpy as np
from ultralytics import YOLO
import ultralytics
ultralytics.checks()

Ultralytics 8.3.122 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 42.1/107.7 GB disk)


# load models

In [7]:
from deep_sort_realtime.deepsort_tracker import DeepSort

# Initialize DeepSort
tracker = DeepSort(max_age=10)

In [8]:
# Your YOLOv8 fine-tuned model
yolo_model = YOLO('/content/drive/MyDrive/icebergtrack/YOLO/best_yolo_0428.pt')  # Update this path if needed

# Initialize DeepSORT
tracker = DeepSort(max_age=30)

# Folder where your frames are stored
frame_folder = '/content/drive/MyDrive/icebergtrack/sentinel1/hh/mark1.0_glac_img/'  # Update this path

# Load all frame paths, sorted
frame_paths = sorted(glob.glob(os.path.join(frame_folder, '*.jpg')))

# Check frame size (assume all frames have same size)
sample_frame = cv2.imread(frame_paths[0])
height, width = sample_frame.shape[:2]

# Optional: Create a VideoWriter to save the result
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('tracked_output.mp4', fourcc, 5, (width, height))

In [ ]:
frame_paths[0][-14:]

'2020-01-01.jpg'

In [ ]:
for frame_path in frame_paths:
    frame = cv2.imread(frame_path)

    # Predict using YOLO
    results = yolo_model.predict(frame, imgsz=640, conf=0.25)

    # Prepare detections for DeepSORT
    detections = []
    if results and len(results[0].boxes.xyxy) > 0:
        for box, conf, cls in zip(results[0].boxes.xyxy, results[0].boxes.conf, results[0].boxes.cls):
            x1, y1, x2, y2 = box.tolist()
            w, h = x2 - x1, y2 - y1
            detections.append(([x1, y1, w, h], conf.item(), int(cls.item())))

    # Update tracker
    tracks = tracker.update_tracks(detections, frame=frame)

    # Draw tracks
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        l, t, r, b = track.to_ltrb()
        l, t, r, b = int(l), int(t), int(r), int(b)

        # Draw bounding box and ID
        cv2.rectangle(frame, (l, t), (r, b), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {track_id}', (l, t - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Save frame to video
    out.write(frame)

out.release()
print("Tracking complete! Saved to tracked_output.mp4")


0: 416x640 1 iceberg, 413.6ms
Speed: 4.4ms preprocess, 413.6ms inference, 1.1ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 3 icebergs, 569.5ms
Speed: 5.6ms preprocess, 569.5ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 (no detections), 388.2ms
Speed: 4.6ms preprocess, 388.2ms inference, 0.7ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 1 iceberg, 404.8ms
Speed: 5.3ms preprocess, 404.8ms inference, 1.1ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 4 icebergs, 392.6ms
Speed: 4.3ms preprocess, 392.6ms inference, 1.1ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 5 icebergs, 615.2ms
Speed: 5.7ms preprocess, 615.2ms inference, 1.5ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 2 icebergs, 632.8ms
Speed: 4.4ms preprocess, 632.8ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 2 icebergs, 620.5ms
Speed: 4.5ms preprocess, 620.5ms inference, 1.5ms postp

In [ ]:
output_dir = "/content/drive/MyDrive/icebergtrack/YOLO/tracked_frames/"
os.makedirs(output_dir, exist_ok=True)

for idx, frame_path in enumerate(frame_paths):
    frame = cv2.imread(frame_path)

    # Predict using YOLO
    results = yolo_model.predict(frame, imgsz=640, conf=0.25)

    # Prepare detections for DeepSORT
    detections = []
    if results and len(results[0].boxes.xyxy) > 0:
        for box, conf, cls in zip(results[0].boxes.xyxy, results[0].boxes.conf, results[0].boxes.cls):
            x1, y1, x2, y2 = box.tolist()
            w, h = x2 - x1, y2 - y1
            detections.append(([x1, y1, w, h], conf.item(), int(cls.item())))

    # Update tracker
    tracks = tracker.update_tracks(detections, frame=frame)

    # Draw tracks
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        l, t, r, b = track.to_ltrb()
        l, t, r, b = int(l), int(t), int(r), int(b)

        # Draw bounding box and ID
        cv2.rectangle(frame, (l, t), (r, b), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {track_id}', (l, t - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Save each frame as an image
    frame_filename = os.path.join(output_dir, frame_path[-14:])
    cv2.imwrite(frame_filename, frame)

print("Tracking complete! Frames saved to tracked_frames/")


0: 416x640 1 iceberg, 547.5ms
Speed: 14.3ms preprocess, 547.5ms inference, 30.8ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 3 icebergs, 455.6ms
Speed: 4.8ms preprocess, 455.6ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 (no detections), 439.0ms
Speed: 4.6ms preprocess, 439.0ms inference, 1.1ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 1 iceberg, 426.2ms
Speed: 4.9ms preprocess, 426.2ms inference, 1.1ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 4 icebergs, 664.2ms
Speed: 4.9ms preprocess, 664.2ms inference, 1.5ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 5 icebergs, 483.6ms
Speed: 5.2ms preprocess, 483.6ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 2 icebergs, 405.3ms
Speed: 6.6ms preprocess, 405.3ms inference, 1.1ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 2 icebergs, 459.0ms
Speed: 5.8ms preprocess, 459.0ms inference, 1.5ms pos

In [9]:
import os
import cv2
import numpy as np
import colorsys

def get_distinct_color(id):
    """Generate a visually distinct color based on HSV space."""
    # Use golden ratio to distribute hues evenly
    golden_ratio_conjugate = 0.618033988749895
    hue = (id * golden_ratio_conjugate) % 1
    r, g, b = colorsys.hsv_to_rgb(hue, 0.7, 0.95)
    return (int(r * 255), int(g * 255), int(b * 255))  # Convert to BGR for OpenCV

output_dir = "/content/drive/MyDrive/icebergtrack/YOLO/tracked_frames/"
os.makedirs(output_dir, exist_ok=True)

for idx, frame_path in enumerate(frame_paths):
    frame = cv2.imread(frame_path)

    # Predict using YOLO
    results = yolo_model.predict(frame, imgsz=640, conf=0.25)

    # Prepare detections for DeepSORT
    detections = []
    if results and len(results[0].boxes.xyxy) > 0:
        for box, conf, cls in zip(results[0].boxes.xyxy, results[0].boxes.conf, results[0].boxes.cls):
            x1, y1, x2, y2 = box.tolist()
            w, h = x2 - x1, y2 - y1
            detections.append(([x1, y1, w, h], conf.item(), int(cls.item())))

    # Update tracker
    tracks = tracker.update_tracks(detections, frame=frame)

    # Draw tracks
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        l, t, r, b = track.to_ltrb()
        l, t, r, b = int(l), int(t), int(r), int(b)

        # Get unique color for each ID
        # color = get_distinct_color(track_id)
        color = get_distinct_color(int(track_id))

        # Draw bounding box and ID
        cv2.rectangle(frame, (l, t), (r, b), color, 2)
        cv2.putText(frame, f'ID: {track_id}', (l, t - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Save frame as image
    frame_filename = os.path.join(output_dir, frame_path[-14:])
    cv2.imwrite(frame_filename, frame)

print("Tracking complete! Frames saved to tracked_frames/")



0: 416x640 1 iceberg, 433.2ms
Speed: 4.3ms preprocess, 433.2ms inference, 1.6ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 3 icebergs, 434.7ms
Speed: 4.8ms preprocess, 434.7ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 (no detections), 423.5ms
Speed: 4.7ms preprocess, 423.5ms inference, 0.7ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 1 iceberg, 414.3ms
Speed: 4.6ms preprocess, 414.3ms inference, 1.1ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 4 icebergs, 414.1ms
Speed: 4.0ms preprocess, 414.1ms inference, 1.8ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 5 icebergs, 891.6ms
Speed: 9.1ms preprocess, 891.6ms inference, 1.7ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 2 icebergs, 424.3ms
Speed: 4.9ms preprocess, 424.3ms inference, 1.1ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 2 icebergs, 400.0ms
Speed: 4.6ms preprocess, 400.0ms inference, 1.2ms postp